In [1]:
import json
import socket
import time
import uuid
from npc_engine.src.config.config import PROJECT_ROOT_PATH

In [2]:
import threading
from npc_engine.src.engine import NPCEngine
def send_data(data, max_packet_size=6000):

        engine_url = "::1"
        engine_port = 8199
        game_url = "::1"
        game_port = 8084

        sock = socket.socket(socket.AF_INET6, socket.SOCK_DGRAM)
        # sock.bind(("::1", game_port))

        # UUID作为消息ID
        msg_id = uuid.uuid4().hex
        # 将json字符串转换为bytes
        data = json.dumps(data).encode('utf-8')
        # 计算数据包总数
        packets = [data[i: i + max_packet_size] for i in range(0, len(data), max_packet_size)]
        total_packets = len(packets)
        for i, packet in enumerate(packets):
            # 构造UDP数据包头部
            #print("sending packet {} of {}, size: {} KB".format(i + 1, total_packets, self.calculate_str_size_in_kb(packet)))
            header = f"{msg_id}@{i + 1}@{total_packets}".encode('utf-8')
            # 发送UDP数据包
            sock.sendto(header + b"@" + packet, (engine_url, engine_port))
        sock.close()

path = PROJECT_ROOT_PATH / "src" / "engine.py"
import os
# os popen
#os.popen("python " + str(path))

In [7]:
from test_config.test_packets import init_packet
def test_engine_init():
    """
    测试引擎初始化
    向引擎发送初始化包，检查引擎是否正确初始化
    初始化包例：
         请参考test_config.test_packets
    :return:
    """
    print(init_packet)
    # 发送初始化包到引擎
    print("sending first")
    send_data(init_packet)
    print("sent first")

test_engine_init()

{'func': 'init', 'scene_name': '雁栖村', 'language': 'C', 'npc': [{'name': '渔夫阿强', 'desc': '渔夫阿强是一个老练的渔民，擅长捕鱼和航海。他有一头浓密的白发和一双狡猾的眼睛。阿强经验丰富，对海洋和天气变化有着敏锐的观察力。', 'mood': '满足', 'npc_state': {'position': '河边钓鱼点', 'observation': {'people': [], 'items': ['船舱', '渔网', '渔具', '航海地图', '渔获'], 'locations': ['船舱内部', '甲板']}, 'backpack': ['鱼饵', '渔具维修工具']}, 'action_space': ['move', 'talk'], 'memory': ['从小就跟随父亲学习捕鱼技巧。', '曾多次出海捕鱼，积累丰富的经验。', '对海洋生态保护有着浓厚的兴趣。', '帮助其他渔民修理损坏的渔具。', '梦想拥有一艘自己的渔船，开展独立的渔业。']}, {'name': '猎人阿明', 'desc': '猎人阿明是一位勇敢而机敏的猎人。他身材魁梧，肌肉发达，眼神犀利。阿明擅长追踪和狩猎各种野生动物，具有过人的耐力和狙击技巧。', 'mood': '专注', 'npc_state': {'position': '猎人小屋', 'observation': {'people': [], 'items': ['猎枪', '弓箭', '追踪装备', '野外求生工具'], 'locations': ['猎人小屋内部', '周围的森林']}, 'backpack': ['干粮', '水壶', '急救包']}, 'action_space': ['move', 'talk'], 'memory': ['从小生活在山区，接受父亲的猎人训练。', '熟悉各种野生动物的习性和行踪。', '常常在附近的森林中追踪并捕获猎物。', '有着长时间在野外生存的经验。', '一日作息：清晨起床后进行锻炼和瞄准训练，白天进行狩猎和追踪，傍晚返回小屋整理装备并准备晚餐，晚上休息并回顾一天的狩猎经历。']}]}
sending first
sent first


In [8]:
from test_config.test_packets import wakeup_packet_test_repeat_move
def test_repeat_move():
    """
    测试引擎初始化
    向引擎发送初始化包，检查引擎是否正确初始化
    初始化包例：
         请参考test_config.test_packets
    :return:
    """
    # 发送初始化包到引擎
    print("sending first")
    send_data(wakeup_packet_test_repeat_move)
    print("sent first")
test_repeat_move()

sending first
sent first


In [8]:
from test_config.test_packets import wakeup_packet_1, wakeup_packet_2, wakeup_packet_3
def test_wake_up():

    """
    测试引擎wake_up函数
    向引擎发送初始化包，检查引擎是否正确初始化
    wakeup包例：
        请参考test_config.test_packets
    预期返回包:
    {
            "name":"action",
            "npc_name":"王大妈",
            "action":"chat",
            "object":"李大爷",
            "parameters":["你吃饭了没？"],
        }
    :return:
    """
    # 发送初始化包到引擎
    print("sending first")
    send_data(wakeup_packet_1)
    print(wakeup_packet_1)
    send_data(wakeup_packet_2)
    print(wakeup_packet_2)
    print("all done")

test_wake_up()

sending first
{'func': 'wake_up', 'npc_name': '王大妈', 'scenario_name': '李大爷家', 'npc_state': {'position': '李大爷家卧室', 'observation': {'people': ['李大爷', '村长', '隐形李飞飞'], 'items': ['椅子#1', '椅子#2', '椅子#3[李大爷占用]', '床[包括:被子、枕头、床单、床垫、私房钱]'], 'locations': ['李大爷家大门', '李大爷家后门', '李大爷家院子']}, 'backpack': ['优质西瓜', '大砍刀', '黄金首饰']}, 'time': '2021-01-01 12:00:00'}
{'func': 'wake_up', 'npc_name': '李大爷', 'scenario_name': '李大爷家', 'npc_state': {'position': '李大爷家卧室', 'observation': {'people': ['王大妈', '村长', '隐形李飞飞'], 'items': ['椅子#1', '椅子#2', '椅子#3[李大爷占用]', '床[包括:被子、枕头、床单、床垫、私房钱]'], 'locations': ['李大爷家大门', '李大爷家后门', '李大爷家院子']}, 'backpack': ['黄瓜', '1000元', '老报纸']}, 'time': '2021-01-01 12:00:00'}
all done


In [5]:
from test_config.test_packets import action_done_packet_1,action_done_packet_2
def test_action_done():
    """
    发送动作完成包到引擎
    GAME发送的包：
    参考test_config.test_packets
    引擎返回的包：
    {
        "func":"action_done",
        "npc_name":"王大妈",
        "action":"chat",
        "object":"李大爷",
        "parameters":["你吃饭了没？"],
    }
    :return:
    """

    print("sending")
    send_data(action_done_packet_1)
    print(action_done_packet_1)
    send_data(action_done_packet_2)
    print(action_done_packet_2)
    print("all done")

test_action_done()

sending
{'func': 'action_done', 'npc_name': '王大妈', 'status': 'success', 'time': '2021-01-01 12:00:00', 'scenario_name': '李大爷家', 'npc_state': {'position': '李大爷家卧室', 'observation': {'people': ['李大爷', '村长', '李飞飞'], 'items': ['椅子#1', '椅子#2', '椅子#3[李大爷占用]', '床'], 'locations': ['李大爷家大门', '李大爷家后门', '李大爷家院子']}, 'backpack': ['优质西瓜', '大砍刀', '黄金首饰']}, 'action': 'mov', 'object': '李大爷家', 'parameters': [], 'reason': ''}
{'func': 'action_done', 'npc_name': '李大爷', 'status': 'fail', 'time': '2021-01-01 12:00:00', 'scenario_name': '李大爷家', 'npc_state': {'position': '李大爷家卧室', 'observation': {'people': ['王大妈', '村长', '李飞飞'], 'items': ['椅子#1', '椅子#2', '椅子#3[李大爷占用]', '床'], 'locations': ['李大爷家大门', '李大爷家后门', '李大爷家院子']}, 'backpack': ['优质西瓜', '大砍刀', '黄金首饰']}, 'action': 'mov', 'object': '警察局', 'parameters': [], 'reason': '李大爷在去往‘警察局’的路上被王大妈打断'}
all done


In [26]:
from test_config.test_packets import action_done_packet_1,action_done_packet_2
def test_action_done():
    """
    发送动作完成包到引擎
    GAME发送的包：
    参考test_config.test_packets
    引擎返回的包：
    {
        "func":"action_done",
        "npc_name":"王大妈",
        "action":"chat",
        "object":"李大爷",
        "parameters":["你吃饭了没？"],
    }
    :return:
    """

    print("sending")
    send_data(action_done_packet_1)
    print(action_done_packet_1)
    send_data(action_done_packet_2)
    print(action_done_packet_2)
    print("all done")

test_action_done()

sending
{'func': 'action_done', 'npc_name': '王大妈', 'status': 'success', 'time': '2021-01-01 12:00:00', 'scenario_name': '李大爷家', 'npc_state': {'position': '李大爷家卧室', 'observation': {'people': ['李大爷', '村长', '李飞飞'], 'items': ['椅子#1', '椅子#2', '椅子#3[李大爷占用]', '床'], 'locations': ['李大爷家大门', '李大爷家后门', '李大爷家院子']}, 'backpack': ['优质西瓜', '大砍刀', '黄金首饰']}, 'action': 'mov', 'object': '李大爷家', 'parameters': [], 'reason': ''}
{'func': 'action_done', 'npc_name': '李大爷', 'status': 'fail', 'time': '2021-01-01 12:00:00', 'scenario_name': '李大爷家', 'npc_state': {'position': '李大爷家卧室', 'observation': {'people': ['王大妈', '村长', '李飞飞'], 'items': ['椅子#1', '椅子#2', '椅子#3[李大爷占用]', '床'], 'locations': ['李大爷家大门', '李大爷家后门', '李大爷家院子']}, 'backpack': ['优质西瓜', '大砍刀', '黄金首饰']}, 'action': 'mov', 'object': '警察局', 'parameters': [], 'reason': '李大爷在去往‘警察局’的路上被王大妈打断'}
all done


In [16]:
from test_config.test_packets import player2npc_packet
from test_config.test_packets import init_packet_police
def test_talk_npc():
    """
    发送player2npc_talk包到引擎
    GAME发送的包：
    参考test_config.test_packets
    引擎返回的包：
    {
        "func":"action_done",
        "npc_name":"王大妈",
        "action":"chat",
        "object":"李大爷",
        "parameters":["你吃饭了没？"],
    }
    :return:
    """

    print("sending")
    send_data(init_packet_police)
    send_data(player2npc_packet)
    print("all done")


test_talk_npc()

sending
all done


In [11]:
from test_config.test_packets import close_packet
def test_close():
    """
    发送关闭包到引擎
    GAME发送的包：
    参考test_config.test_packets
    引擎返回的包：
    {
        "func":"close"
    }
    :return:
    """

    print("sending")
    send_data(close_packet)
    print(close_packet)
    print("all done")

test_close()

sending
{'func': 'close'}
all done
